# Instructions:
This notebook makes the analysis of large volumes of results easier. It simply displays in a human-friendly way the values of decision variables (first part) or KPIs (second parts). Here is a quick starter guide:
1. Set the path to your ResultsBatch instance in the `Results loading` cell
2. Either:
    - run all cells (menu > cells > run all)
    - run at least `Imports`, `Results loading` and `Tools`. Then run the cells you are interested in.
3. Interact with the cell output. Figures will Update automatically if the `Update` field is on `True`.
   You can select multiple choices for values that are not related to energy systems description, i.e. Hub, Production, Storage, etc...
   
notes: 
- each cell is independant from the other.
- not all KPIs are shown, but the main are

# <font color=blue> Imports [run me]</font>

In [ ]:
import os
from sys import path
from pathlib import Path
           
from tamos.data_IO import ResultsBatch

import pandas as pd
from numpy import nan, inf, stack
from numpy import min as min_
from functools import reduce

from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import ipywidgets as widgets

import plotly 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "notebook"
import tqdm as tqdm

idx = pd.IndexSlice

# <font color=blue> Results loading [run me] </font> 

In [ ]:
# The following snippet load a ResultsBatch instance that was written on disk using ResultsBatch.dump_object().
RB = ResultsBatch.load_object("")
vars_, KPIs, results = RB.vars_, RB.KPIs, RB.results

# <font color=blue> Tools [run me]</font>

In [ ]:
to_list = lambda x: x if isinstance(x, (tuple, list)) else [x]
remove_unique_choice = True

def subset_df(df, subset_mapper, recursive=True):
    if recursive:
        if subset_mapper:
            col, values = subset_mapper.popitem()
            df_ = df[df[col].isin(to_list(values))]
            return subset_df(df_, subset_mapper, True)
        else:
            return df    
    else:
        cond = reduce(lambda x, y: x & y, [df[col].isin(to_list(values)) 
                                           for col, values in subset_mapper.items()])
        return df[cond]

def get_widgets(regulars, df_regulars, time_series=False):
    """
    Regulars: non-descriptors columns for which widgets are created
    """
    helper_widgets = {"Update": widgets.Select(options=[True, False], 
                                               value=False, 
                                               description="Update", 
                                               disabled=False, rows=2)}
    if time_series:
        helper_widgets["Mode"] = widgets.Select(options=['Time series', 'Duration', 'Describe'],
                                                   description='Mode',
                                                   disabled=False,
                                                rows=3
                                                   )

    descriptors_widgets = {k: widgets.Select(options=sorted(v, key=repr), 
                                               description=k, 
                                               disabled=False,
                                               rows=min(len(v), 8))
                           for k, v in RB.relevant_descriptors.items()}
    regular_widgets = {}
    for col in regulars:
        options = sorted(df_regulars[col].unique())
        regular_widgets[col] = widgets.SelectMultiple(options=options, 
                                                   description=col, 
                                                   disabled=False,
                                                  rows=min(len(options), 8))
    
    all_widgets = {**helper_widgets, **descriptors_widgets, **regular_widgets}
    if remove_unique_choice:
        all_widgets = {k: v for k, v in all_widgets.items() if len(v.options)>1}
    
    return all_widgets

# <font color=blue> Figures </font>

## <font color=red > Variables </font> 

### note: List of relevant columns per variable

In [ ]:
pd.Series({var_name: list(var.columns.difference(RB.relevant_descriptors)) for var_name, var in vars_.items()}).sort_index()

### SP_P [Production]

In [ ]:
widgets_ = get_widgets(["Hub", "Production"], vars_["SP_P"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(vars_["SP_P"], widgets__)
        fig = px.bar(df_, y="Production", x="SP_P", color="Hub", barmode='group', labels={"SP_P": "Capacity (kW)"})
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### Q_P [Production]

In [ ]:

widgets_ = get_widgets(["Hub", "Production"], vars_["Q_P"], time_series=True) 

def f(**widgets__):    
    if widgets__.pop("Update"):      
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["Q_P"], widgets__)
        labels ={"Q_P": "Main production (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="Q_P", facet_row="Production", color="Hub", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="Q_P", facet_row="Production", color="Hub", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:
            df__ = df_.groupby(["Production", "Hub"])["Q_P"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                         facet_row="Production", color="Hub", barmode="group") 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### F_P [Production]

In [ ]:
widgets_ = get_widgets(["Hub", "Production", "Element"], vars_["F_P"], True) 

def f(**widgets__):    
    if widgets__.pop("Update"):        
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["F_P"], widgets__)
        labels ={"F_P": "Incoming power (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="F_P", facet_row="Production", facet_col="Hub", color="Element", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="F_P", facet_row="Production", facet_col="Hub", color="Element", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:            
            df__ = df_.groupby(["Production", "Hub", "Element"])["F_P"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                         facet_row="Production", facet_col="Hub", color="Element", barmode="group") 
      
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()

ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### SE_S [Storage]

In [ ]:
widgets_ = get_widgets(["Hub"], vars_["SE_S"])
def f(**widgets__):
    if widgets__.pop("Update"):        
        df_ = subset_df(vars_["SE_S"], widgets__).dropna()
        fig = px.bar(df_, y="Storage", x="SE_S", color="Hub", barmode='group', labels={"SE_S": "Capacity (kg or kWh)"})
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### E_S [Storage]

In [ ]:
widgets_ = get_widgets(["Hub", "Storage"], vars_["E_S"], True)
def f(**widgets__):
    if widgets__.pop("Update"):        
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["E_S"], widgets__)
        labels={"E_S": "Stage of charge (kg or kWh)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="E_S", facet_row="Storage", color="Hub", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="E_S", facet_row="Storage", color="Hub", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:
            df__ = df_.groupby(["Storage", "Hub"])["E_S"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                         facet_row="Storage", color="Hub", barmode="group") 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### F_S [Storage]

In [ ]:
widgets_ = get_widgets(["Hub", "Storage"], vars_["F_S"], True)
def f(**widgets__):
    if widgets__.pop("Update"):        
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["F_S"], widgets__)
        labels={"F_S": "Incoming power (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="F_S", facet_row="Storage", facet_col="Hub", color="Element", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="F_S", facet_row="Storage", facet_col="Hub", color="Element", labels=labels,
                          ecdfmode="complementary",
                          ecdfnorm=None)
        else:
            df__ = df_.groupby(["Storage", "Hub"])["F_S"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value", facet_row="Storage", facet_col="Hub", color="Element", 
                         barmode="group") 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### F_EXT [ElementIO]

In [ ]:
widgets_ = get_widgets(["Hub", "ElementIO"], vars_["F_EXT"], True)
def f(**widgets__):
    if widgets__.pop("Update"):  
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["F_EXT"], widgets__)
        labels={"F_EXT": "Incoming power (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="F_EXT", facet_row="ElementIO", color="Hub", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="F_EXT", facet_row="ElementIO", color="Hub", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:
            df__ = df_.groupby(["ElementIO", "Hub"])["F_EXT"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                         facet_row="ElementIO", color="Hub", barmode="group") 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### F_SYS [Network]

In [ ]:
widgets_ = get_widgets(["Hub", "Network"], vars_["F_SYS"], True)
def f(**widgets__):
    if widgets__.pop("Update"):                
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["F_SYS"], widgets__)
        labels={"F_SYS": "Incoming power (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="F_SYS", facet_row="Network", color="Hub", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="F_SYS", facet_row="Network", color="Hub", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:            
            df__ = df_.groupby(["Network", "Hub"])["F_SYS"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                         facet_row="Network", color="Hub", barmode="group")
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### F_N [Network]

In [ ]:
widgets_ = get_widgets(["Hub", "Hub_bis", "Network"], vars_["F_N"], True)
def f(**widgets__):
    if widgets__.pop("Update"):                
        mode = widgets__.pop("Mode")
        df_ = subset_df(vars_["F_N"], widgets__)
        labels = {"F_N": "Power from Hub to Hub_bis (kW)"}
        if mode == "Time series":
            fig = px.line(df_, x="Date", y="F_N", facet_row="Hub", facet_col="Hub_bis", color="Network", labels=labels)
        elif mode == "Duration":
            fig = px.ecdf(df_, y="F_N", facet_row="Hub", facet_col="Hub_bis", color="Network", labels=labels,
                         ecdfmode="complementary",
                         ecdfnorm=None)
        else:
            
            df__ = df_.groupby(["Network", "Hub", "Hub_bis"])["F_N"].describe()
            df__ = df__.melt(ignore_index=False).reset_index()
            fig = px.bar(df__, x="variable", y="value",
                          facet_row="Hub", facet_col="Hub_bis", color="Network", barmode="group") 
        fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
        fig.show()
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

### X_N [Network]

In [ ]:
widgets_ = get_widgets(["Network"], vars_["X_N"], False)
def f(**widgets__):
    if widgets__.pop("Update"):             
        df_ = subset_df(vars_["X_N"], widgets__)
        labels = {"F_N": "Power from Hub to Hub_bis (kW)"}
        df_ = df_.pivot_table(index="Hub", columns="Hub_bis", values="X_N")
        df_ = df_.where(df_==1, nan)
        df_ = df_.dropna(axis=1, how="all").dropna(axis=0, how="all").fillna("")
        df_ = df_.replace({1: "X"})
        display(df_)
    
           
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_))
display(ui, out)

## <font color=red> KPIs </font>

### note: List of relevant columns per KPI

In [ ]:
pd.Series({KPI_name: list(KPI.columns.difference(RB.relevant_descriptors)) for KPI_name, KPI in KPIs.items()}).sort_index()

### COST_production

In [ ]:
widgets_ = get_widgets(["Hub"], KPIs["COST_production"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["COST_production"], widgets__)
        fig = px.bar(df_, y="Production", x="COST_production", color="Hub", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### COST_element

In [ ]:
widgets_ = get_widgets(["Hub"], KPIs["COST_element"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["COST_element"], widgets__)
        fig = px.bar(df_, y="ElementIO", x="COST_element", color="Hub", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### COST_network

In [ ]:
widgets_ = get_widgets([], KPIs["COST_network"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["COST_network"], widgets__)
        fig = px.bar(df_, y="Network", x="COST_network", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### COST_storage

In [ ]:
widgets_ = get_widgets(["Hub"], KPIs["COST_storage"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["COST_storage"], widgets__)
        fig = px.bar(df_, y="Storage", x="COST_storage", color="Hub", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### ElementIO CO2

In [ ]:
widgets_ = get_widgets(["Hub"], KPIs["ElementIO CO2"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["ElementIO CO2"], widgets__)
        fig = px.bar(df_, y="ElementIO", x="ElementIO CO2", color="Hub", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)

### ElementIO exergy

In [ ]:
widgets_ = get_widgets(["Hub"], KPIs["ElementIO Exergy"]) 
def f(**widgets__):    
    if widgets__.pop("Update"):        
        df_ = subset_df(KPIs["ElementIO Exergy"], widgets__)
        fig = px.bar(df_, y="ElementIO", x="ElementIO Exergy", color="Hub", barmode='group')
        fig.show()
ui = widgets.Box(children=list(widgets_.values()))
out = interactive_output(f, dict(widgets_)) # copy
display(ui, out)